In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
import datetime
warnings.filterwarnings('ignore')

## Cardiogenic Shock

In [2]:
#IMPORT FILES NEEDED
shockOrig = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Cardiogenic Shock Full Data Set.csv", sep=",", index_col='PAT_ID').sort_index()
cath = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Cath Data.csv", sep=",", index_col='PAT_ID').sort_index()
echo = pd.read_csv("../Data Validation/Cardiogenic Shock/Raw Data/Echo Data.csv", sep=",", index_col='PAT_ID').sort_index()


In [3]:
# # Find indexes from shock that are not cath
# np.setdiff1d(shockOrig.index, cath.index)

In [4]:
#For initial training, just use first row for each patient
shock = pd.DataFrame(columns=shockOrig.columns)
print("Num indexes = ", len(set(shockOrig.index)))

for idx in set(shockOrig.index):
    pt = shockOrig.loc[idx]
    if type(pt) != pd.Series:
#         print(pt)
        row = pt.iloc[0]
        shock = shock.append(row)
    else:
        shock = shock.append(pt)

shockOrig = shock.sort_index()

shockOrig

Num indexes =  401


,PAT_SEX,PAT_DOB,IndexAdmissionCSN,IndexAdmitDt,IndexDischDt,Deceased,H2H,SystolicHF,DiastolicHF,HeartTx,...,PA_DIA,PA_MEAN,PCW_MEAN,FICK_HR,FICKCO,FICKCI,TDCO,TDCI,PA_SAT,DeathDate
Z1002839,Male,2/25/59,2.000000e+11,8/7/17,8/13/17,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8/13/17
Z101592,Male,7/3/38,2.000000e+11,4/13/18,4/19/18,1.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6/8/18
Z101721,Male,10/13/57,2.000000e+11,10/31/17,11/11/17,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/11/17
Z1017923,Male,6/7/56,2.000000e+11,11/20/18,12/7/18,1.0,1.0,1.0,NaN,NaN,...,20,26,15,99,4.83,1.87,7.5,2.91,55.1,7/30/20
Z1024230,Male,11/21/55,2.000000e+11,4/6/18,4/19/18,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z95784,Female,9/2/33,7.013672e+07,1/15/17,2/10/17,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2/10/17
Z96296,Female,10/2/48,2.000000e+11,4/1/19,4/10/19,1.0,NaN,NaN,1.0,NaN,...,11,18,7,70,2.32,1.44,3.5,2.17,58.4,12/24/19
Z964794,Male,8/18/55,7.213152e+07,4/12/17,4/27/17,1.0,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/27/17
Z977261,Female,11/16/71,2.000000e+11,10/11/19,10/18/19,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/18/19


In [5]:
#Make date columns date objects
cath['ProcDate'] = pd.to_datetime(cath['ProcDate'])
echo['Result_Date'] = pd.to_datetime(echo['Result_Date'])
shockOrig['PAT_DOB'] = pd.to_datetime(shockOrig['PAT_DOB'])
shockOrig['IndexAdmitDt'] = pd.to_datetime(shockOrig['IndexAdmitDt'])
shockOrig['CATH_DATE'] = pd.to_datetime(shockOrig['CATH_DATE'])
shockOrig['ADM1_AdmitDt'] = pd.to_datetime(shockOrig['ADM1_AdmitDt'])
shockOrig['ADM2_AdmitDt'] = pd.to_datetime(shockOrig['ADM2_AdmitDt'])
shockOrig['ADM3_AdmitDt'] = pd.to_datetime(shockOrig['ADM3_AdmitDt'])

#sort cath by index and date
cath = cath.sort_values(['PAT_ID', 'ProcDate'], ascending=[True, True])

#NOTE - TODO Keep this?????
#drop patients with heart TX
for idx in shockOrig.index:
    pt = shockOrig.loc[idx]
    
    if pt['HeartTx'] == 1:
        shockOrig = shockOrig.drop([idx])
        
        if idx in cath.index:
            cath = cath.drop([idx])
            

cath.loc['Z2642077']
# cath.loc['Z137291']
cath

,ProcDate,PAT_ENC_CSN_ID,Flo_Disply_Nme,Msr_Val,FlowsheetDate,Msr_Rec_Tm,Msr_Line,Msr_Rec_Dtm,Msr_Time,PhaseName
PAT_ID,,,,,,,,,,
Z1017923,2018-11-26,2.000000e+11,Thermal CI,2.91,11/26/18,10:12:55,527,12:55.0,12:55.0,Room Air Rest
Z1017923,2018-11-26,2.000000e+11,Fick HR,99.00,11/26/18,10:12:55,520,12:55.0,12:55.0,Room Air Rest
Z1017923,2018-11-26,2.000000e+11,PA Sat,55.10,11/26/18,10:12:55,528,12:55.0,12:55.0,Room Air Rest
Z1017923,2018-11-26,2.000000e+11,Thermal CO,7.50,11/26/18,10:12:55,526,12:55.0,12:55.0,Room Air Rest
Z1017923,2018-11-26,2.000000e+11,PA Dia,20.00,11/26/18,11:29:28,488,29:28.0,29:28.0,Room Air Rest
...,...,...,...,...,...,...,...,...,...,...
Z96296,2019-10-30,2.000000e+11,Fick HR,70.00,10/30/19,13:01:50,839,01:50.0,01:50.0,Room Air Rest
Z96296,2019-10-30,2.000000e+11,Fick CO,2.32,10/30/19,13:01:50,853,01:50.0,01:50.0,Room Air Rest
Z96296,2019-10-30,2.000000e+11,Fick CI,1.44,10/30/19,13:01:50,854,01:50.0,01:50.0,Room Air Rest


# HEMO

In [6]:
#go thru and append multiple rows as needed
giantList = []

for idx in shockOrig.index:
    lst = []
    if idx not in cath.index: #verifying no hemo recorded in shock dataset that not in cath
        pt = shockOrig.loc[idx]
        if type(pt) != pd.Series:
            pt = pt.iloc[0]
        if not pd.isna(pt['RA_MEAN']) or not pd.isna(pt['PA_SYS']) or not pd.isna(pt['TDCO']):
                print(idx, "-Found hemo not in cath dataset")
    
    else:
        pt = cath.loc[idx]
#         print(idx)
        
        #get age, gender, race
        shPt = shockOrig.loc[idx]
        dob = shPt['PAT_DOB']
        admDate = shPt['IndexAdmitDt']
        age = admDate.year - dob.year - ((admDate.month, admDate.day) < (dob.month, dob.day))
        print(age)
        
        if shPt['PAT_SEX'] == 'Male': 
            gender = 1
        else:
            gender = 2
            
        race = np.nan
        
        #Ejf for dif admissions
        ef = shPt['ADM1_DischargeEF']
        if not pd.isna(ef):
            adm1EF = re.findall(r'\d+', ef)
            adm1EF = [float(i) for i in adm1EF]
            if len(adm1EF) > 1:
                adm1EF = sum(adm1EF) / len(adm1EF)
            else:
                adm1EF = adm1EF[0]
        else:
            adm1EF = np.nan
            
        ef = shPt['ADM2_DischargeEF']
        if not pd.isna(ef):
            adm2EF = re.findall(r'\d+', ef)
            adm2EF = [float(i) for i in adm2EF]
            if len(adm2EF) > 1:
                adm2EF = sum(adm2EF) / len(adm2EF)
            else:
                adm2EF = adm2EF[0]
        else:
            adm2EF = np.nan

        ef = shPt['ADM3_DischargeEF']
        if not pd.isna(ef):
            adm3EF = re.findall(r'\d+', ef)
            adm3EF = [float(i) for i in adm3EF]
            if len(adm3EF) > 1:
                adm3EF = sum(adm3EF) / len(adm3EF)
            else:
                adm3EF = adm3EF[0]
        else:
            adm3EF = np.nan

        dates = sorted(list(set(pt['ProcDate'])))
        for d in dates: #iterate through each timestamp w/recorded hemo
            ptDt = pt[pt['ProcDate'] == d]
            
            #get apropriate ejf and bps
            adms = []
            if not pd.isnull(shPt['ADM1_AdmitDt']):
                adms.append(shPt['ADM1_AdmitDt'])
            if not pd.isnull(shPt['ADM2_AdmitDt']):
                adms.append(shPt['ADM2_AdmitDt'])
            if not pd.isnull(shPt['ADM3_AdmitDt']):
                adms.append(shPt['ADM3_AdmitDt'])
            
            closest = min(adms, key=lambda x: abs(x - d))
            if closest == shPt['ADM1_AdmitDt']:
                ejf = adm1EF
                bp = shPt['ADM1_SysBP']
                if not pd.isna(bp):
                    vals = str(bp).split('/')
                    sys = float(vals[0])
                    dias = float(vals[1])
                else:
                    sys = np.nan
                    dias = np.nan
            elif closest == shPt['ADM2_AdmitDt']:
                ejf = adm2EF
                bp = shPt['ADM2_SysBP']
                if not pd.isna(bp):
                    vals = str(bp).split('/')
                    sys = float(vals[0])
                    dias = float(vals[1])
                else:
                    sys = np.nan
                    dias = np.nan
            else:
                ejf = adm3EF
                bp = shPt['ADM3_SysBP']
                if not pd.isna(bp):
                    vals = str(bp).split('/')
                    sys = float(vals[0])
                    dias = float(vals[1])
                else:
                    sys = np.nan
                    dias = np.nan
            
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'RA Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                rap = val[0]
            elif len(val) > 1:
                rap = sum(val) / len(val)
            else: #val == none
                rap = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Sys']['Msr_Val'].tolist()
            if len(val) == 1:
                pas = val[0]
            elif len(val) > 1:
                pas = sum(val) / len(val)
            else: #val == none
                pas = np.nan
                
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Dia']['Msr_Val'].tolist()
            if val == []: 
                    val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA DIa']['Msr_Val'].tolist()

            if len(val) == 1:
                pad = val[0]
            elif len(val) > 1:
                pad = sum(val) / len(val)
            else: #val == none
                pad = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                pamn = val[0]
            elif len(val) > 1:
                pamn = sum(val) / len(val)
            else: #val == none
                pamn = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PCW Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                pcwp = val[0]
            elif len(val) > 1:
                pcwp = sum(val) / len(val)
            else: #val == none
                pcwp = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Thermal CO']['Msr_Val'].tolist()
            if val == []:
                val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick CO']['Msr_Val'].tolist()
            if len(val) == 1:
                co = val[0]
            elif len(val) > 1:
                co = sum(val) / len(val)
            else: #val == none
                co = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Thermal CI']['Msr_Val'].tolist()
            if val == []:
                val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick CI']['Msr_Val'].tolist()
            if len(val) == 1:
                ci = val[0]
            elif len(val) > 1:
                ci = sum(val) / len(val)
            else: #val == none
                ci = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Sat']['Msr_Val'].tolist()
            if len(val) == 1:
                mixed = val[0]
            elif len(val) > 1:
                mixed = sum(val) / len(val)
            else: #val == none
                mixed = np.nan
                
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick HR']['Msr_Val'].tolist()
            if len(val) == 1:
                hr = val[0]
            elif len(val) > 1:
                hr = sum(val) / len(val)
            else: #val == none
                hr = np.nan
                
            #Composite Hemo
            MAP = sys + ((2 * dias) / 3)
            MPAP = pas + ((2 * pad) / 3)
            CPI = (ci * MAP) / 451
            PP = sys - dias
            PPP = PP / sys
            PAPP = (pas  - pad) / pas
            SVR = 80 * (MAP  - rap) / co
            RAT = rap / pcwp
            PPRatio = PP / hr
            PAPi = (pas - pad) / rap if rap else 0
            SAPi = (sys - dias) / pcwp
            CPP = dias - pcwp
            PRAPRat = PP / rap if rap else 0

            lst = [idx, age, gender, race, ejf, rap, pas, pad, pamn, pcwp, co, ci, mixed, sys, dias, hr, MAP, MPAP, CPI,
                  PP, PPP, PAPP, SVR, RAT, PPRatio, PAPi, SAPi, CPP, PRAPRat]
            
            giantList.append(lst)
            
            

            
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
hemoDF

-38
-41
-35
37
-39
-30
-28
-14
-49
-30
-43
-20
-30
-32
-41
-39
44
-27
-48
27
-25
-19
-45
40
-39
-50
-32
-53
26
-33
-41
-50
-40
-17
38
-40
-29
-47
-52
-47
-52
21
-24
-46
-29
38
36
-48
-28
-41
-34
-43
-51
-37
-49
-45
-32
40
-18
48
26
-37
35
-33
39
43
42
-51
-27
-44
-39
-47
45
39
-45
-51
-19
44
-38
31
-24
-22
-43
-39
-35
-38
-12
-47
-27
-47
43
-34
41
-44
-35
44
-31
-52
-43
21
42
-29
-25
45
42
-45
-29
-27
-26
-29
31
-50
40
35
-41
-33
-39
-44
-32
-41
-26
-23
42
-32
-30
-24
-47
-49
-29
-30


,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
Z1017923,-38,1,NaN,22.5,9.0,38.0,20.0,26.0,15.0,7.50,...,31.0,0.303922,0.473684,1496.888889,0.600000,0.313131,2.000000,2.066667,56.0,3.444444
Z1024990,-41,1,NaN,17.5,12.0,51.0,33.0,39.0,31.0,5.67,...,70.0,0.555556,0.352941,2135.214580,0.387097,0.608696,1.500000,2.258065,25.0,5.833333
Z104044,-35,1,NaN,NaN,12.0,61.8,21.6,36.6,26.0,4.13,...,53.0,0.445378,0.650485,2924.939467,0.461538,NaN,3.350000,2.038462,40.0,4.416667
Z1099417,37,2,NaN,27.5,32.0,58.0,34.0,44.0,21.0,3.39,...,41.0,0.445652,0.413793,2218.289086,1.523810,0.369369,0.750000,1.952381,30.0,1.281250
Z1140320,-39,2,NaN,27.5,19.0,54.0,24.0,38.0,36.0,3.27,...,37.0,0.352381,0.555556,3213.047910,0.527778,0.430233,1.578947,1.027778,32.0,1.947368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z928524,-49,1,NaN,15.0,16.0,85.0,29.0,50.0,24.0,7.13,...,46.0,0.368000,0.658824,1813.931744,0.666667,NaN,3.500000,1.916667,55.0,2.875000
Z928524,-49,1,NaN,27.5,21.0,62.0,32.0,42.0,23.0,2.03,...,56.0,0.478632,0.483871,5385.878489,0.913043,0.848485,1.428571,2.434783,38.0,2.666667
Z928524,-49,1,NaN,27.5,23.0,67.0,35.0,45.0,21.0,4.20,...,56.0,0.478632,0.477612,2565.079365,1.095238,0.543689,1.391304,2.666667,40.0,2.434783


In [7]:
#save to file
hemoDF.to_csv("Preprocessed Data/HemoCardiogenicShock.csv")


In [8]:
#Make Hemo Labels
labels = pd.DataFrame()
indexes = sorted(list(set(hemoDF.index)))
labels['ID'] = indexes
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in indexes:
    pt = shockOrig.loc[idx]
    if pt['Deceased'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

labels

,Death
ID,
Z1017923,1
Z1024990,1
Z104044,1
Z1099417,0
Z1140320,1
...,...
Z888391,1
Z927126,1
Z928524,1


In [9]:
labels.to_csv("Preprocessed Data/HemoLabelsCardiogenicShock.csv")

# All Data

In [10]:
shock = shockOrig

cohort = pd.DataFrame()
cohort['ID'] = shock.index
cohort = cohort.set_index('ID')

lst = []
for idx in shock.index: #age at first admission
    pt = shock.loc[idx]
    
    dob = pt['PAT_DOB']
    admDate = pt['IndexAdmitDt']
    
    age = admDate.year - dob.year - ((admDate.month, admDate.day) < (dob.month, dob.day))
    lst.append(age)

cohort['Age'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['PAT_SEX'] == 'Male': 
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#will have 3 rows --> adm1, adm2 and adm3
cohort['BMI_adm1'] = shock['ADM1_BMI'].tolist()
cohort['BMI_adm2'] = shock['ADM2_BMI'].tolist()
cohort['BMI_adm3'] = shock['ADM3_BMI'].tolist()

#convert ejf from ranges to values for 3 adms
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM1_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
    lst.append(val)
        
cohort['EjF_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM2_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
    lst.append(val)
        
cohort['EjF_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM3_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
    lst.append(val)
        
cohort['EjF_adm3'] = lst


#Initial Hosp Days
cohort['InitialHospDays_adm1'] = shock['ADM1_LOS'].tolist()
cohort['InitialHospDays_adm2'] = shock['ADM2_LOS'].tolist()
cohort['InitialHospDays_adm3'] = shock['ADM3_LOS'].tolist()

#Total Hosp days
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    a1 = pt['ADM1_LOS']
    a2 = pt['ADM2_LOS']
    a3 = pt['ADM3_LOS']
    if pd.isna(pt['ADM1_LOS']):
        a1 = 0
    if pd.isna(pt['ADM2_LOS']):
        a2 = 0
    if pd.isna(pt['ADM3_LOS']):
        a3 = 0
    
    los = a1 + a2 + a3
    lst.append(los)

cohort['TotalHospDays'] = lst
    
cohort.head(15)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays
ID,,,,,,,,,,,,
Z1002839,-42,1,NaN,NaN,NaN,17.5,NaN,NaN,6,NaN,NaN,6.0
Z101592,-21,1,19.3,18.3,21.1,22.5,NaN,NaN,6,3.0,8.0,17.0
Z101721,-40,1,NaN,29.0,31.8,22.5,NaN,42.5,11,2.0,9.0,22.0
Z1017923,-38,1,34.5,31.6,NaN,22.5,47.5,NaN,17,9.0,NaN,26.0
Z1024230,-38,1,25.9,NaN,NaN,37.5,NaN,NaN,13,4.0,NaN,17.0
Z1024990,-41,1,26.2,NaN,NaN,17.5,NaN,NaN,3,NaN,NaN,3.0
Z104044,-35,1,NaN,NaN,NaN,22.5,NaN,NaN,5,3.0,2.0,10.0
Z1078182,-39,1,28.8,NaN,NaN,17.5,NaN,NaN,20,NaN,NaN,20.0
Z107856,-46,2,36.4,NaN,NaN,67.5,NaN,NaN,55,3.0,2.0,60.0


In [11]:
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['COPD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['DM'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['CKD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['HeartTx'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

cohort.head(5)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays,COPD,DIAB,RENAL,HTRANS
ID,,,,,,,,,,,,,,,,
Z1002839,-42,1,NaN,NaN,NaN,17.5,NaN,NaN,6,NaN,NaN,6.0,0,0,0,0
Z101592,-21,1,19.3,18.3,21.1,22.5,NaN,NaN,6,3.0,8.0,17.0,1,0,0,0
Z101721,-40,1,NaN,29.0,31.8,22.5,NaN,42.5,11,2.0,9.0,22.0,0,1,0,0
Z1017923,-38,1,34.5,31.6,NaN,22.5,47.5,NaN,17,9.0,NaN,26.0,0,1,1,0
Z1024230,-38,1,25.9,NaN,NaN,37.5,NaN,NaN,13,4.0,NaN,17.0,0,1,1,0


In [12]:
# Labs
cohort['BUN_adm1'] = shock['ADM1_Labs_BUN'].tolist()
cohort['BUN_adm2'] = shock['ADM2_Labs_BUN'].tolist()
cohort['BUN_adm3'] = shock['ADM3_Labs_BUN'].tolist()

cohort['CRT_adm1'] = shock['ADM1_Labs_Cr'].tolist()
cohort['CRT_adm2'] = shock['ADM2_Labs_Cr'].tolist()
cohort['CRT_adm3'] = shock['ADM3_Labs_Cr'].tolist()

cohort['SOD_adm1'] = shock['ADM1_Labs_NA'].tolist()
cohort['SOD_adm2'] = shock['ADM2_Labs_NA'].tolist()
cohort['SOD_adm3'] = shock['ADM3_Labs_NA'].tolist()

cohort['WBC_adm1'] = shock['ADM1_Labs_Lymphocytes'].tolist()
cohort['WBC_adm2'] = shock['ADM2_Labs_Lymphocytes'].tolist()
cohort['WBC_adm3'] = shock['ADM3_Labs_Lymphocytes'].tolist()

#Meds
#ACE
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM1_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM1_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM2_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM2_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM3_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM3_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm3'] = lst

#BET
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm3'] = lst

#DIUR
cohort['DIUR_adm1'] = shock['ADM1_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm2'] = shock['ADM2_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm3'] = shock['ADM3_Diuretics__Potassium Sparing Diuretics'].tolist()

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,WBC_adm3,ACE_adm1,ACE_adm2,ACE_adm3,BET_adm1,BET_adm2,BET_adm3,DIUR_adm1,DIUR_adm2,DIUR_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,-42,1,NaN,NaN,NaN,17.5,NaN,NaN,6,NaN,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
Z101592,-21,1,19.3,18.3,21.1,22.5,NaN,NaN,6,3.0,...,0.93,0,0,1,0,0,0,NaN,NaN,1.0
Z101721,-40,1,NaN,29.0,31.8,22.5,NaN,42.5,11,2.0,...,NaN,0,1,1,0,0,1,NaN,NaN,1.0
Z1017923,-38,1,34.5,31.6,NaN,22.5,47.5,NaN,17,9.0,...,NaN,0,0,0,0,1,0,NaN,NaN,NaN
Z1024230,-38,1,25.9,NaN,NaN,37.5,NaN,NaN,13,4.0,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z946846,42,2,NaN,NaN,NaN,57.5,NaN,NaN,6,NaN,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
Z95784,-17,2,34.7,33.3,NaN,52.5,57.5,NaN,26,4.0,...,NaN,0,1,0,0,0,0,NaN,NaN,NaN
Z96296,-30,2,27.6,NaN,NaN,67.5,57.5,62.5,9,8.0,...,0.82,1,1,1,0,0,0,NaN,NaN,NaN


In [13]:
#Invasive Hemo
# cohort['HRTRT_adm1'] = shock['ADM1_EKG_QRSD'].tolist()
# cohort['HRTRT_adm2'] = shock['ADM2_EKG_QRSD'].tolist()
# cohort['HRTRT_adm3'] = shock['ADM3_EKG_QRSD'].tolist()

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM1_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM2_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM3_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm3'] = lst

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM1_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm1'] = sys
cohort['BPDIAS_adm1'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM2_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm2'] = sys
cohort['BPDIAS_adm2'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM3_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm3'] = sys
cohort['BPDIAS_adm3'] = dias

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,DIUR_adm3,HRTRT_adm1,HRTRT_adm2,HRTRT_adm3,BPSYS_adm1,BPDIAS_adm1,BPSYS_adm2,BPDIAS_adm2,BPSYS_adm3,BPDIAS_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,-42,1,NaN,NaN,NaN,17.5,NaN,NaN,6,NaN,...,NaN,93.0,NaN,NaN,91.0,49.0,NaN,NaN,NaN,NaN
Z101592,-21,1,19.3,18.3,21.1,22.5,NaN,NaN,6,3.0,...,1.0,121.0,120.0,119.0,100.0,54.0,90.0,52.0,118.0,60.0
Z101721,-40,1,NaN,29.0,31.8,22.5,NaN,42.5,11,2.0,...,1.0,115.0,101.0,107.0,83.0,73.0,NaN,NaN,145.0,70.0
Z1017923,-38,1,34.5,31.6,NaN,22.5,47.5,NaN,17,9.0,...,NaN,112.0,NaN,NaN,102.0,71.0,134.0,76.0,NaN,NaN
Z1024230,-38,1,25.9,NaN,NaN,37.5,NaN,NaN,13,4.0,...,NaN,115.0,NaN,NaN,93.0,56.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z946846,42,2,NaN,NaN,NaN,57.5,NaN,NaN,6,NaN,...,NaN,76.0,NaN,NaN,130.0,65.0,NaN,NaN,NaN,NaN
Z95784,-17,2,34.7,33.3,NaN,52.5,57.5,NaN,26,4.0,...,NaN,98.0,80.0,NaN,103.0,44.0,121.0,60.0,NaN,NaN
Z96296,-30,2,27.6,NaN,NaN,67.5,57.5,62.5,9,8.0,...,NaN,127.0,122.0,124.0,107.0,52.0,146.0,56.0,131.0,63.0


In [14]:
#Composite Hemo

idx = sorted(set(cohort.index))

#MAP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]
    
    admit1 = t['BPSYS_adm1'] + ((2 * t['BPDIAS_adm1']) / 3)
    admit2 = t['BPSYS_adm2'] + ((2 * t['BPDIAS_adm2']) / 3)
    admit3 = t['BPSYS_adm3'] + ((2 * t['BPDIAS_adm3']) / 3)

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['MAP_adm1'] = a1
cohort['MAP_adm2'] = a2
cohort['MAP_adm3'] = a3
    
#PP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['BPSYS_adm1'] - t['BPDIAS_adm1']
    admit2 = t['BPSYS_adm2'] - t['BPDIAS_adm2']
    admit3 = t['BPSYS_adm3'] - t['BPDIAS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PP_adm1'] = a1
cohort['PP_adm2'] = a2
cohort['PP_adm3'] = a3

#PPP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['BPSYS_adm1']
    admit2 = t['PP_adm2'] / t['BPSYS_adm2']
    admit3 = t['PP_adm3'] / t['BPSYS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPP_adm1'] = a1
cohort['PPP_adm2'] = a2
cohort['PPP_adm3'] = a3

#PPRatio
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['HRTRT_adm1']
    admit2 = t['PP_adm2'] / t['HRTRT_adm2']
    admit3 = t['PP_adm3'] / t['HRTRT_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPRatio_adm1'] = a1
cohort['PPRatio_adm2'] = a2
cohort['PPRatio_adm3'] = a3

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,MAP_adm3,PP_adm1,PP_adm2,PP_adm3,PPP_adm1,PPP_adm2,PPP_adm3,PPRatio_adm1,PPRatio_adm2,PPRatio_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,-42,1,NaN,NaN,NaN,17.5,NaN,NaN,6,NaN,...,NaN,42.0,NaN,NaN,0.461538,NaN,NaN,0.451613,NaN,NaN
Z101592,-21,1,19.3,18.3,21.1,22.5,NaN,NaN,6,3.0,...,158.000000,46.0,38.0,58.0,0.460000,0.422222,0.491525,0.380165,0.316667,0.487395
Z101721,-40,1,NaN,29.0,31.8,22.5,NaN,42.5,11,2.0,...,191.666667,10.0,NaN,75.0,0.120482,NaN,0.517241,0.086957,NaN,0.700935
Z1017923,-38,1,34.5,31.6,NaN,22.5,47.5,NaN,17,9.0,...,NaN,31.0,58.0,NaN,0.303922,0.432836,NaN,0.276786,NaN,NaN
Z1024230,-38,1,25.9,NaN,NaN,37.5,NaN,NaN,13,4.0,...,NaN,37.0,NaN,NaN,0.397849,NaN,NaN,0.321739,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z946846,42,2,NaN,NaN,NaN,57.5,NaN,NaN,6,NaN,...,NaN,65.0,NaN,NaN,0.500000,NaN,NaN,0.855263,NaN,NaN
Z95784,-17,2,34.7,33.3,NaN,52.5,57.5,NaN,26,4.0,...,NaN,59.0,61.0,NaN,0.572816,0.504132,NaN,0.602041,0.762500,NaN
Z96296,-30,2,27.6,NaN,NaN,67.5,57.5,62.5,9,8.0,...,173.000000,55.0,90.0,68.0,0.514019,0.616438,0.519084,0.433071,0.737705,0.548387


In [15]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row2 = []
    row0.append(df.name)
    row1.append(df.name)
    row2.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
            row2.append(df[col])
        elif col + "_adm1" in cohort.columns and col + "_adm2" in cohort.columns and col + "_adm3" in cohort.columns:
            row0.append(df[col+"_adm1"])
            row1.append(df[col+"_adm2"])
            row2.append(df[col+"_adm3"])
        else:
            row0.append(np.nan)
            row1.append(np.nan)
            row2.append(np.nan)
    
    giantList.append(row0)
    giantList.append(row1)
    giantList.append(row2)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
Z1002839,-42.0,1.0,NaN,NaN,NaN,6.0,6.0,NaN,NaN,NaN,...,NaN,17.5,49.0,91.0,NaN,NaN,123.666667,42.0,0.461538,0.451613
Z1002839,-42.0,1.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1002839,-42.0,1.0,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z101592,-21.0,1.0,NaN,NaN,19.3,6.0,17.0,NaN,NaN,NaN,...,NaN,22.5,54.0,100.0,NaN,NaN,136.000000,46.0,0.460000,0.380165
Z101592,-21.0,1.0,NaN,NaN,18.3,3.0,17.0,NaN,NaN,NaN,...,NaN,NaN,52.0,90.0,NaN,NaN,124.666667,38.0,0.422222,0.316667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z964794,-39.0,1.0,NaN,NaN,NaN,5.0,33.0,NaN,NaN,NaN,...,1.0,NaN,61.0,94.0,NaN,NaN,134.666667,33.0,0.351064,0.358696
Z964794,-39.0,1.0,NaN,NaN,28.0,13.0,33.0,NaN,NaN,NaN,...,NaN,15.0,57.0,91.0,NaN,NaN,129.000000,34.0,0.373626,0.253731
Z977261,47.0,2.0,NaN,NaN,38.6,7.0,7.0,NaN,NaN,NaN,...,NaN,47.5,55.0,112.0,NaN,NaN,148.666667,57.0,0.508929,0.527778


In [16]:
#save to file
allData.to_csv("Preprocessed Data/AllDataCardiogenicShock.csv")

In [17]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = shock.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['Deceased'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst


labels

,Death
ID,
Z1002839,1
Z101592,1
Z101721,1
Z1017923,1
Z1024230,0
...,...
Z946846,0
Z95784,1
Z96296,1


In [18]:
labels.to_csv("Preprocessed Data/LabelsCardiogenicShock.csv")